# Initialize imports

In [ ]:
from classes import *
from functions import *
import pandas as pd
import numpy as np

# Setting up Y_Bus, bus and line objects for all task

In [ ]:
line_data = ReadCsvFile('./files/given_network/network_configuration_line_data.csv')
bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack1.csv')
# bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack2.csv')

Sbase = 100 # MVA
Ubase = 230 # kV
Zbase = (Ubase**2)/Sbase
max_iterations = 300
tolerance = 1e-3
Q_lim = -0.6
V_lim = -0.1 # take 1 - V_lim for max and 1 - abs(V_lim) for min

# Newton Raphson Load Flow Algorithm

In [ ]:
from NRLF import NewtonRaphson

NewtonRaphson(bus_data=bus_data, 
              line_data=line_data, 
              Sbase=Sbase, 
              max_iterations=max_iterations, 
              tolerance=tolerance, 
              Q_lim=Q_lim,
              V_lim=V_lim
              )


## Output format
X<br>Bus no.|Bus Information<br>Volts (pu)|<br>Angle (deg)|Generation<br>(MW)|Generation<br>(MVAr)|Load<br>(MW)|Load<br>(MVAr)|<br>Bus type|<br>To Bus no.|Line Flow<br>(MW)|Line Flow<br>(MVAr)|
---|---|---|---|---|---|---|---|---|---|---
|1|1.06|0|-|-|0|0|SLACK|2<br>3|||
|2|||40|-|20|10|PV|1<br>3<br>4<br>5|||
|3|||0|0|45|15|PQ|1<br>2<br>4|||
|4|||0|0|40|5|PQ|2<br>3<br>5|||
|5|||0|0|60|10|PQ|2<br>4|||


# Decoupled Load Flow

In [ ]:
from DLF import DLF

DLF(bus_data=bus_data, 
    line_data=line_data, 
    Sbase=Sbase, 
    max_iterations=max_iterations, 
    tolerance=tolerance, 
    Q_lim=Q_lim,
    V_lim=V_lim
    )


# Fast Decoupled Load Flow

In [ ]:
from FDLF import FDLF

FDLF(bus_data=bus_data, 
    line_data=line_data, 
    Sbase=Sbase, 
    max_iterations=max_iterations, 
    tolerance=tolerance
    )

# DC Power Flow

In [ ]:
from DCPF import DCPF

DCPF(bus_data=bus_data,
     line_data=line_data,
     Sbase=Sbase
     )